In [ ]:
!pip install bing-image-downloader

In [ ]:
!mkdir Dataset

In [ ]:
from bing_image_downloader import downloader as dl

nd = ["Gambar Banjir","Gambar Gempa Bumi","Gambar Tsunami","Gambar tanah longsor"]
dl.download(nd[0],limit=500,output_dir='Dataset')
dl.download(nd[1],limit=500,output_dir='Dataset')
dl.download(nd[2],limit=500,output_dir='Dataset')
dl.download(nd[3],limit=500,output_dir='Dataset')

Streaming output truncated to the last 5000 lines.
[%] Indexed 3 Images on Page 2110.




[!!]Indexing page: 2111

[%] Indexed 4 Images on Page 2111.




[!!]Indexing page: 2112

[%] Indexed 9 Images on Page 2112.




[!!]Indexing page: 2113

[%] Indexed 4 Images on Page 2113.




[!!]Indexing page: 2114

[%] Indexed 4 Images on Page 2114.




[!!]Indexing page: 2115

[%] Indexed 9 Images on Page 2115.




[!!]Indexing page: 2116

[%] Indexed 4 Images on Page 2116.




[!!]Indexing page: 2117

[%] Indexed 4 Images on Page 2117.




[!!]Indexing page: 2118

[%] Indexed 4 Images on Page 2118.




[!!]Indexing page: 2119

[%] Indexed 4 Images on Page 2119.




[!!]Indexing page: 2120

[%] Indexed 9 Images on Page 2120.




[!!]Indexing page: 2121

[%] Indexed 4 Images on Page 2121.




[!!]Indexing page: 2122

[%] Indexed 4 Images on Page 2122.




[!!]Indexing page: 2123

[%] Indexed 4 Images on Page 2123.




[!!]Indexing page: 2124

[%] Indexed 9 Images on Page 2124.




[!!]Indexing p

In [ ]:
dl.download(nd[1],limit=1000,output_dir='Dataset')


In [ ]:
dl.download(nd[2],limit=1000,output_dir='Dataset')


In [ ]:
dl.download(nd[3],limit=1000,output_dir='Dataset')


# Model Building